## Processing T1w images to estimate Gray Matter Density


In [6]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber, FreeSurferSource
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm.preprocess import VBMSegment, Segment
from nipype.interfaces.ants import Atropos, Registration, ApplyTransforms, N4BiasFieldCorrection
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer import FSCommand, Normalize
from pandas import DataFrame, Series, read_csv

# FSL setup- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study specific variables
study_home = '/moochie/user_data/CamachoCat/Aggregate_anats/GMD_ML'

#subject_info = read_csv(study_home + '/doc/subject_info.csv', index_col=None)
#subjects_list = subject_info['freesurferID'].tolist()
subjects_list = ['C1034']

raw_data = study_home + '/raw'
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'

sample_template = study_home + '/templates/lcbd_template_1mm.nii.gz'
sample_template_brain = study_home + '/templates/lcbd_template_1mm_brain.nii.gz'
sample_template_mask = study_home + '/templates/lcbd_template_1mm_mask.nii.gz'

# tissue priors
csf_prior = study_home + '/templates/tissue_priors/tissue0.nii.gz'
wm_prior = study_home + '/templates/tissue_priors/tissue3.nii.gz'
gm_prior = study_home + '/templates/tissue_priors/gm_tissue.nii.gz'

#freesurfer setup
fs_dir = '/moochie/user_data/CamachoCat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

In [7]:
# Data handling nodes
infosource = Node(IdentityInterface(fields=['subjid']), 
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

fs_source = Node(FreeSurferSource(subjects_dir=fs_dir), 
                 name='fs_source')

substitutions = [('_subjid_', '')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=output_dir,
                         container=output_dir), 
                name='datasink')

### Bias-correct and segment subject images

In [8]:
def smooth_tissue(tissue_files):
    def brightthresh(img):
        import nibabel as nib
        from numpy import median, where

        from nipype import config, logging
        config.enable_debug_mode()
        logging.update_logging(config)

        img_nifti1 = nib.load(img)
        img_data = img_nifti1.get_data()
        img_data = img_data.astype(float)

        brain_values = where(img_data > 0)
        median_thresh = median(brain_values)
        bright_thresh = 0.75 * median_thresh

        return(bright_thresh)
    
    from nipype.interfaces.fsl import SUSAN, Threshold
    sm = SUSAN()
    sm.inputs.fwhm=4
    thr = Threshold()
    thr.inputs.thresh=0.05
    thr.inputs.direction='below'

    for file in tissue_files:
        if 'POSTERIOR_02' in file:
            sm.inputs.brightness_threshold = brightthresh(file)
            sm.inputs.in_file = file
            sm.inputs.out_file = file.replace('POSTERIOR_02','smoothed_gm')
            sm.run()
            thr.inputs.in_file = sm.inputs.out_file
            thr.inputs.out_file = file.replace('POSTERIOR_02','final_smooth_gm')
            thr.run()
            smoothed_gm = file.replace('POSTERIOR_02','final_smooth_gm')
        
    return(smoothed_gm)

In [9]:
# N4 bias correction
biascorrect = Node(N4BiasFieldCorrection(save_bias=True),name='biascorrect')

# normalize white matter intensities using control points
norm_intensity = Node(Normalize(subjects_dir=fs_dir, out_file='norm_T1.nii.gz', args='-mprage'),
                      name='norm_intensity')

# Register to template brain
reg_temp1 = Node(Registration(fixed_image=sample_template,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=4,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Rigid', 'Affine'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp1')

# Skullstripping
skullstrip = Node(BET(),name='skullstrip')

# Register to skullstripped brain
reg_temp2 = Node(Registration(fixed_image=sample_template_brain,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=4,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Affine', 'SyN'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp2')

# Atropos segmentation
segment = Node(Atropos(initialization='PriorProbabilityImages',
                       prior_probability_images=[wm_prior, gm_prior, csf_prior],
                       number_of_tissue_classes=3, 
                       prior_weighting = 0.5,
                       prior_probability_threshold = 0.75,
                       num_threads =4,
                       mask_image=sample_template_mask, 
                       save_posteriors=True, n_iterations=10),name='segment')

smooth = Node(Function(input_names=['tissue_files'], 
                       output_names=['smoothed_gm'], 
                       function=smooth_tissue), name='smooth')

In [10]:
segproc = Workflow(name='segproc')
segproc.connect([(infosource, fs_source,[('subjid','subject_id')]),
                 (fs_source,biascorrect,[('rawavg','input_image')]),
                 (biascorrect, norm_intensity, [('output_image','in_file')]),
                 (norm_intensity, reg_temp1,[('out_file','moving_image')]),
                 (reg_temp1, skullstrip, [('warped_image','in_file')]),
                 (skullstrip, reg_temp2, [('out_file','moving_image')]), 
                 (reg_temp2, segment, [('warped_image','intensity_images')]),
                 (segment, smooth, [('posteriors','tissue_files')]),
                 (reg_temp1, datasink, [('composite_transform','reg1_composite_transform'), 
                                        ('inverse_composite_transform','reg1_inverse_composite_transform'),
                                        ('warped_image','reg1_warped_anat')]),
                 (reg_temp2, datasink, [('composite_transform','reg2_composite_transform'),
                                        ('inverse_composite_transform','reg2_inverse_composite_transform'),
                                        ('warped_image','reg2_warped_anat')]),
                 (segment, datasink, [('classified_image','hard_tissue_files'),('posteriors','soft_tissue_files')]),
                 (smooth, datasink, [('smoothed_gm','final_gmd')])
                 
                ])
segproc.base_dir = workflow_dir
segproc.write_graph(graph2use='flat')
segproc.run('MultiProc', plugin_args={'n_procs': 24,'memory_gb':30})

190518-10:51:56,937 workflow INFO:
	 Generated workflow graph: /moochie/user_data/CamachoCat/Aggregate_anats/GMD_ML/workflows/segproc/graph.png (graph2use=flat, simple_form=True).
190518-10:51:57,124 workflow INFO:
	 Workflow segproc settings: ['check', 'execution', 'logging', 'monitoring']
190518-10:51:57,234 workflow INFO:
	 Running in parallel.
190518-10:51:57,238 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 30.00/30.00, Free processors: 24/24.
190518-10:51:57,416 workflow INFO:
	 [Node] Setting-up "segproc.fs_source" in "/moochie/user_data/CamachoCat/Aggregate_anats/GMD_ML/workflows/segproc/_subjid_C1034/fs_source".
190518-10:51:57,789 workflow INFO:
	 [Node] Running "fs_source" ("nipype.interfaces.io.FreeSurferSource")
190518-10:51:58,47 workflow INFO:
	 [Node] Finished "segproc.fs_source".
190518-10:51:59,240 workflow INFO:
	 [Job 0] Completed (segproc.fs_source).
190518-10:51:59,245 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs rea